# Implementation example

In [8]:
#Imports
import numpy as np
import pandas as pd

import sys
sys.path.insert(0, './esnet/')
from esnet import *

from esnet.optimizers import GradientOptimizer
from esnet import PredictionESN

import matplotlib.pyplot as plt

This Jupyter Notebook's code implements an example of the use of a ESN package called ESN-Easy. It proceeds as the following:
1. Imports the easyESN library and utils needed
2. Takes the provided dataframe of inputs and targets (see column headings in csv). Call this dataframe pred.
3. Accepts the dataframe, columns for targets, columns for predictions as inputs. Then, it splits the data into three categories : train, validation, test.
4. Recreates a network until it has a the best MSE value (Loops until a good enough ESN is built).
5. Predicts the next 2 points for the "Target1" & "Input6" column and return it as a list for each one

In [9]:
path = "./inputdata.csv"
pred = pd.read_csv(path)

This function partitions the data into train, test & validation sets. It requires the user to set the 'train_percent' which basically the percentage of data that will be used as training set, and also 'validate_percent' which represents the percentage of the data that will be used as validation set. The remaining data will be used as test set.

In [10]:
def train_validate_test_split(df, train_percent=.8, validate_percent=.15, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

This function compares the MSE of the 1st generated model with maxMSE value, then, the keeps comparing the MSE of several models with different parameters and select the best performing one for the final prediction.

In [11]:
def NextNValues(df,inputColumns, targetColumns, predictAhead, maxMSE):
    #df : The dataframe 
    #inputColumns : the input columns
    #targetColumns : the target columns 
    #predictAhead : the number of values to predict 
    #maxMSE = 0.5,   The maximum MSE value.  
    #                
    
    num_reservoir = [50, 75, 100, 150, 200, 250]
    leakingrate = [0.025, 0.05, 0.1, 0.2, 0.3, 0.4]
    regressionparameters = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
    
    train, validate, test = train_validate_test_split(df)
    
    inputDataTraining = train[inputColumns]
    outputDataTraining = train[targetColumns]
    
    inputDataValidation = validate[inputColumns]
    outputDataValidation = validate[targetColumns]    
    
    inputDataTest = test[inputColumns]
    outputDataTest = test[targetColumns]    
    
    num_input = len(inputColumns)
    num_output = len(targetColumns)
    
    esn_ret = PredictionESN(n_input=num_input, n_output=num_output, n_reservoir=50, leakingRate=0.001, 
                        regressionParameters=[1e-5], solver="lsqr", feedback=False)
    
    num_res = num_reservoir
    leaking_rate_ = leakingrate
    
    count=0
    
    MSE = maxMSE
    
    best_num_res = 0
    best_leakingrate = 0
    best_earning_rates = 0
    
    arr = list()
    for i in range(len(targetColumns)):
        arr.append(outputDataValidation[targetColumns[i]].values)

    for res in num_reservoir:
        for leakingrate_ in leakingrate:
            esn = PredictionESN(n_input=num_input, n_output=num_output, n_reservoir=res, 
                                leakingRate=leakingrate_, regressionParameters=regressionparameters)
            
            fit_value = esn.fit(inputDataTraining, outputDataTraining, transientTime="Auto", verbose=0)
            
#             for lr in learning_rates:
#                 opt = GradientOptimizer(esn, learningRate=lr)
#                 validationLosses, fitLosses, penalties, learningRates = opt.optimizePenaltyForEvaluationError(inputDataTraining.values, outputDataTraining.values, inputDataValidation.values, outputDataValidation.values, epochs=num_epochs, transientTime=100, verbose=0)
#                 prediction_val = esn.predict(inputDataValidation)

#             prediction_val = esn.predict(inputDataValidation)
#             print(prediction_val)
#             MSE_val = np.mean((prediction_val - outputDataValidation.values)**2)
            
            prediction_val = esn.predict(inputDataValidation)
                
#             print(outputDataValidation[targetColumns[0]].values)
            MSE_val = np.mean((prediction_val - arr)**2)
            
            if MSE_val < MSE: 
                count += 1
                esn_ret = esn
                MSE = MSE_val 
                best_num_res = res
                best_leakingrate = leakingrate_
                
#   
#     print('Best parameters: ')
#     print('Reservoir: ', best_num_res)
#     print('Leaking rate: ', best_num_res)
#     print('Learning rate: ', best_learning_rates)
#     print('MSE: ', MSE)
    
    generated_ = esn_ret.generate(NextValues=predictAhead, n=len(inputDataValidation), inputData=inputDataValidation, initialOutputData=inputDataTraining.iloc[-1].values)
    
    if len(targetColumns)==1:
        res = generated_[0]
        res = res[:predictAhead]
    elif len(targetColumns)==2:
        res_1 = generated_[0]
        res_1 = res_1[:predictAhead]
        res_2 = generated_[2]
        res_2 = res_2[:predictAhead]
        res=[res_1, res_2]
    elif len(targetColumns)==3:
        res_1 = generated_[0]
        res_1 = res_1[:predictAhead]
        res_2 = generated_[1]
        res_2 = res_2[:predictAhead]
        res_3 = generated_[2]
        res_3 = res_3[:predictAhead]
        res=[res_1, res_2, res_3]
        
    return res

In [12]:


NextNValues(pred,['Input1','Input2','Inpit3','Input4'], ['Target1', "Input6"], 2, 0.5)

[array([1., 1.]), [64.423116, 17.174506]]